In [ ]:
!python --version

# MySQL connection with jupyter notebook

### install mysql-connector-python

In [ ]:
#mysql connection with python
!conda install -c anaconda mysql-connector-python -y

### Connection with mysql

In [ ]:
try:
    import mysql.connector as sql
    #connection to SQL
    conn = sql.connect(
        host='localhost', 
        user='root', 
        password='cap4770',
        database= 'cap4770',  
        use_pure = True)
    
    cursor = conn.cursor(buffered=True)
    cursor.execute("use cap4770")
    
    # Check what tables are available
    cursor.execute("show tables")
    tables = cursor.fetchall()
    print("Available tables in the database:")
    for table in tables:
        print(f"- {table[0]}")
    
except Exception as e:
    print(f"Database connection error: {e}")
    print("Will work with CSV file instead.")

In [ ]:
# install numpy, pandas and matplotlib using pip at terminal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Storm Events Data Analysis

Let's explore the storm events data from the CSV file and database.

In [ ]:
# If the data has been uploaded to the database, query it from there
# First, let's reconnect to the database
try:
    import mysql.connector as sql
    conn = sql.connect(
        host='localhost', 
        user='root', 
        password='cap4770',
        database= 'cap4770',  
        use_pure = True)
    
    cursor = conn.cursor(buffered=True)
    cursor.execute("use cap4770")
    
    # Check what tables are available
    cursor.execute("show tables")
    tables = cursor.fetchall()
    print("Available tables in the database:")
    for table in tables:
        print(f"- {table[0]}")
    
except Exception as e:
    print(f"Database connection error: {e}")
    print("Will work with CSV file instead.")

In [ ]:
# Query storm events data from database (assuming table name is 'storm_events' or similar)
# You may need to adjust the table name based on how you uploaded the data

try:
    # Try common table names for storm events data - add your table name here
    possible_table_names = ['storm_events', 'stormevents', 'storm_data', 'StormEvents_details', 'cap4']
    
    storm_table_name = None
    for table_name in possible_table_names:
        try:
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            count = cursor.fetchone()[0]
            storm_table_name = table_name
            print(f"Found storm events table: {table_name} with {count} records")
            break
        except:
            continue
    
    if storm_table_name:
        # Query the first 10 records from the database
        query = f"SELECT * FROM {storm_table_name} LIMIT 10"
        storm_df_db = pd.read_sql(query, conn)
        print(f"\nData from database table '{storm_table_name}':")
        print(f"Shape: {storm_df_db.shape}")
        storm_df_db.head()
    else:
        print("Storm events table not found in database. Please check the table name.")
        
except Exception as e:
    print(f"Error querying database: {e}")
    print("Working with CSV file data instead.")

In [ ]:

#searches database for the cost of damages from each weather event type and adds them together. Typecasts the damage_property and damage_crops
query = f"SELECT event_type, SUM(CASE WHEN damage_property LIKE '%k' THEN CAST(REPLACE(damage_property, 'k', '') AS DECIMAL(10,2)) * 1000 WHEN damage_property LIKE '%M' THEN CAST(REPLACE(damage_property, 'M', '') AS DECIMAL(10,2)) * 1000000 WHEN damage_property LIKE '%B' THEN CAST(REPLACE(damage_property, 'B', '') AS DECIMAL(10,2)) * 1000000000 ELSE CAST(damage_property AS DECIMAL(10,2)) END + CASE WHEN damage_crops LIKE '%k' THEN CAST(REPLACE(damage_crops, 'k', '') AS DECIMAL(10,2)) * 1000 WHEN damage_crops LIKE '%M' THEN CAST(REPLACE(damage_crops, 'M', '') AS DECIMAL(10,2)) * 1000000 WHEN damage_crops LIKE '%B' THEN CAST(REPLACE(damage_crops, 'B', '') AS DECIMAL(10,2)) * 1000000000 ELSE CAST(damage_crops AS DECIMAL(10,2)) END) AS total FROM {storm_table_name} GROUP BY event_type"
damage_db = pd.read_sql(query, conn)
# Sort by the 'total' column in descending order
damage_db = damage_db.sort_values(by='total', ascending=False)
#Formats cost into a readable format
pd.options.display.float_format = '${:,.0f}'.format
print(damage_db)


# Data Visualizations

Now let's create various graphs to visualize the storm events data from our complete database of 79,091 records.

In [ ]:
# Install additional visualization libraries
!pip install seaborn plotly

In [ ]:
# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set style for better looking plots
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# Graph 1: Total Damage Costs by Event Type (Bar Chart)
plt.figure(figsize=(15, 10))

# Get top 15 most damaging event types for better readability
top_damage = damage_db.head(15)

# Create bar plot
bars = plt.bar(range(len(top_damage)), top_damage['total'], 
               color=plt.cm.viridis(np.linspace(0, 1, len(top_damage))))

# Customize the plot
plt.title('Top 15 Weather Events by Total Damage Cost (2011)', fontsize=16, fontweight='bold')
plt.xlabel('Weather Event Type', fontsize=12)
plt.ylabel('Total Damage Cost (USD)', fontsize=12)
plt.xticks(range(len(top_damage)), top_damage['event_type'], rotation=45, ha='right')

# Format y-axis to show values in millions/billions
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M' if x < 1e9 else f'${x/1e9:.1f}B'))

# Add value labels on top of bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + max(top_damage['total'])*0.01,
             f'${height/1e6:.1f}M' if height < 1e9 else f'${height/1e9:.1f}B',
             ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.grid(axis='y', alpha=0.3)
plt.show()

print(f"Total damage across all event types: ${damage_db['total'].sum():,.0f}")

In [ ]:
# Graph 2: Event Frequency Analysis
# Get event frequency data from database
query_freq = f"SELECT event_type, COUNT(*) as frequency FROM {storm_table_name} GROUP BY event_type ORDER BY frequency DESC"
event_frequency = pd.read_sql(query_freq, conn)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Bar chart of top 15 most frequent events
top_freq = event_frequency.head(15)
bars1 = ax1.bar(range(len(top_freq)), top_freq['frequency'], 
                color=plt.cm.plasma(np.linspace(0, 1, len(top_freq))))
ax1.set_title('Top 15 Most Frequent Weather Events (2011)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Weather Event Type', fontsize=12)
ax1.set_ylabel('Number of Events', fontsize=12)
ax1.set_xticks(range(len(top_freq)))
ax1.set_xticklabels(top_freq['event_type'], rotation=45, ha='right')

# Add value labels
for i, bar in enumerate(bars1):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + max(top_freq['frequency'])*0.01,
             f'{int(height):,}', ha='center', va='bottom', fontsize=10)

ax1.grid(axis='y', alpha=0.3)

# Pie chart of top 10 events
top_10_freq = event_frequency.head(10)
others_freq = event_frequency.iloc[10:]['frequency'].sum()
pie_data = list(top_10_freq['frequency']) + [others_freq]
pie_labels = list(top_10_freq['event_type']) + ['Others']

ax2.pie(pie_data, labels=pie_labels, autopct='%1.1f%%', startangle=90)
ax2.set_title('Distribution of Weather Events (2011)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Total number of weather events: {event_frequency['frequency'].sum():,}")
print(f"Most common event: {event_frequency.iloc[0]['event_type']} ({event_frequency.iloc[0]['frequency']:,} occurrences)")

In [ ]:
# Graph 3: Monthly Timeline of Weather Events
# Get monthly event data
query_monthly = f"""
SELECT 
    MONTH(STR_TO_DATE(BEGIN_DATE_TIME, '%d-%b-%y %H:%i:%s')) as month,
    MIN(MONTHNAME(STR_TO_DATE(BEGIN_DATE_TIME, '%d-%b-%y %H:%i:%s'))) as month_name,
    COUNT(*) as event_count,
    SUM(CASE 
        WHEN damage_property LIKE '%k' THEN CAST(REPLACE(damage_property, 'k', '') AS DECIMAL(10,2)) * 1000 
        WHEN damage_property LIKE '%M' THEN CAST(REPLACE(damage_property, 'M', '') AS DECIMAL(10,2)) * 1000000 
        WHEN damage_property LIKE '%B' THEN CAST(REPLACE(damage_property, 'B', '') AS DECIMAL(10,2)) * 1000000000 
        ELSE CAST(damage_property AS DECIMAL(10,2)) 
    END + CASE 
        WHEN damage_crops LIKE '%k' THEN CAST(REPLACE(damage_crops, 'k', '') AS DECIMAL(10,2)) * 1000 
        WHEN damage_crops LIKE '%M' THEN CAST(REPLACE(damage_crops, 'M', '') AS DECIMAL(10,2)) * 1000000 
        WHEN damage_crops LIKE '%B' THEN CAST(REPLACE(damage_crops, 'B', '') AS DECIMAL(10,2)) * 1000000000 
        ELSE CAST(damage_crops AS DECIMAL(10,2)) 
    END) as total_damage
FROM {storm_table_name} 
GROUP BY month
ORDER by month
"""
monthly_data = pd.read_sql(query_monthly, conn)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

# Event count by month
ax1.plot(monthly_data['month_name'], monthly_data['event_count'], 
         marker='o', linewidth=3, markersize=8, color='steelblue')
ax1.fill_between(monthly_data['month_name'], monthly_data['event_count'], alpha=0.3, color='steelblue')
ax1.set_title('Number of Weather Events by Month (2011)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Month', fontsize=12)
ax1.set_ylabel('Number of Events', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Add value labels
for i, (month, count) in enumerate(zip(monthly_data['month_name'], monthly_data['event_count'])):
    ax1.annotate(f'{count:,}', (i, count), textcoords="offset points", xytext=(0,10), ha='center')

# Total damage by month
bars2 = ax2.bar(monthly_data['month_name'], monthly_data['total_damage'], 
                color=plt.cm.autumn(np.linspace(0, 1, len(monthly_data))))
ax2.set_title('Total Damage Cost by Month (2011)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Month', fontsize=12)
ax2.set_ylabel('Total Damage (USD)', fontsize=12)
ax2.tick_params(axis='x', rotation=45)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.0f}M' if x < 1e9 else f'${x/1e9:.1f}B'))

# Add value labels on bars
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + max(monthly_data['total_damage'])*0.01,
             f'${height/1e6:.0f}M' if height < 1e9 else f'${height/1e9:.1f}B',
             ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

print("Monthly Summary:")
for _, row in monthly_data.iterrows():
    print(f"{row['month_name']}: {row['event_count']:,} events, ${row['total_damage']:,.0f} damage")

In [ ]:
# Graph 4: Interactive Plotly Visualization - Damage vs Frequency Scatter Plot
# Merge damage and frequency data
damage_freq = pd.merge(damage_db, event_frequency, on='event_type', how='inner')

# Create interactive scatter plot
fig = px.scatter(damage_freq, 
                 x='frequency', 
                 y='total', 
                 size='total',
                 hover_name='event_type',
                 title='Weather Events: Damage Cost vs Frequency (2011)',
                 labels={
                     'frequency': 'Number of Events',
                     'total': 'Total Damage Cost (USD)',
                     'event_type': 'Event Type'
                 },
                 color='total',
                 color_continuous_scale='Viridis')

# Update layout
fig.update_layout(
    width=800,
    height=600,
    title_font_size=16,
    showlegend=False
)

# Format hover template
fig.update_traces(
    hovertemplate='<b>%{hovertext}</b><br>' +
                  'Events: %{x:,}<br>' +
                  'Total Damage: $%{y:,.0f}<br>' +
                  '<extra></extra>'
)

fig.show()

# Show summary statistics
print("Correlation Analysis:")
correlation = damage_freq['frequency'].corr(damage_freq['total'])
print(f"Correlation between frequency and damage: {correlation:.3f}")
print(f"This indicates a {'strong' if abs(correlation) > 0.7 else 'moderate' if abs(correlation) > 0.3 else 'weak'} {'positive' if correlation > 0 else 'negative'} relationship.")

In [ ]:
# Graph 5: Geographic Distribution by State
# Get state-wise data
query_states = f"""
SELECT 
    state,
    COUNT(*) as event_count,
    SUM(CASE 
        WHEN damage_property LIKE '%k' THEN CAST(REPLACE(damage_property, 'k', '') AS DECIMAL(10,2)) * 1000 
        WHEN damage_property LIKE '%M' THEN CAST(REPLACE(damage_property, 'M', '') AS DECIMAL(10,2)) * 1000000 
        WHEN damage_property LIKE '%B' THEN CAST(REPLACE(damage_property, 'B', '') AS DECIMAL(10,2)) * 1000000000 
        ELSE CAST(damage_property AS DECIMAL(10,2)) 
    END + CASE 
        WHEN damage_crops LIKE '%k' THEN CAST(REPLACE(damage_crops, 'k', '') AS DECIMAL(10,2)) * 1000 
        WHEN damage_crops LIKE '%M' THEN CAST(REPLACE(damage_crops, 'M', '') AS DECIMAL(10,2)) * 1000000 
        WHEN damage_crops LIKE '%B' THEN CAST(REPLACE(damage_crops, 'B', '') AS DECIMAL(10,2)) * 1000000000 
        ELSE CAST(damage_crops AS DECIMAL(10,2)) 
    END) as total_damage
FROM {storm_table_name} 
GROUP BY state 
ORDER BY total_damage DESC
"""
state_data = pd.read_sql(query_states, conn)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Top 15 states by damage
top_states_damage = state_data.head(15)
bars1 = ax1.barh(range(len(top_states_damage)), top_states_damage['total_damage'], 
                 color=plt.cm.coolwarm(np.linspace(0, 1, len(top_states_damage))))
ax1.set_title('Top 15 States by Total Damage Cost (2011)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Total Damage (USD)', fontsize=12)
ax1.set_ylabel('State', fontsize=12)
ax1.set_yticks(range(len(top_states_damage)))
ax1.set_yticklabels(top_states_damage['state'])
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.0f}M' if x < 1e9 else f'${x/1e9:.1f}B'))

# Add value labels
for i, bar in enumerate(bars1):
    width = bar.get_width()
    ax1.text(width + max(top_states_damage['total_damage'])*0.01, bar.get_y() + bar.get_height()/2,
             f'${width/1e6:.0f}M' if width < 1e9 else f'${width/1e9:.1f}B',
             ha='left', va='center', fontsize=10)

ax1.grid(axis='x', alpha=0.3)

# Top 15 states by event count
top_states_count = state_data.nlargest(15, 'event_count')
bars2 = ax2.barh(range(len(top_states_count)), top_states_count['event_count'], 
                 color=plt.cm.viridis(np.linspace(0, 1, len(top_states_count))))
ax2.set_title('Top 15 States by Number of Events (2011)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Number of Events', fontsize=12)
ax2.set_ylabel('State', fontsize=12)
ax2.set_yticks(range(len(top_states_count)))
ax2.set_yticklabels(top_states_count['state'])

# Add value labels
for i, bar in enumerate(bars2):
    width = bar.get_width()
    ax2.text(width + max(top_states_count['event_count'])*0.01, bar.get_y() + bar.get_height()/2,
             f'{int(width):,}', ha='left', va='center', fontsize=10)

ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("Top 5 States Summary:")
for i, row in state_data.head(5).iterrows():
    print(f"{row['state']}: {row['event_count']:,} events, ${row['total_damage']:,.0f} damage")

In [ ]:
# Show monthly fatalities (direct/indirect)
import calendar

query_fatalities_by_month = f"""
SELECT
    MONTH(STR_TO_DATE(BEGIN_DATE_TIME, '%d-%b-%y %H:%i:%s')) AS month_num,
    SUM(DEATHS_DIRECT) AS direct_deaths,
    SUM(DEATHS_INDIRECT) AS indirect_deaths
FROM {storm_table_name}
WHERE BEGIN_DATE_TIME IS NOT NULL
GROUP BY month_num
ORDER BY month_num
"""

fatalities_monthly = pd.read_sql(query_fatalities_by_month, conn)

# Clean and plot
fatalities_monthly = fatalities_monthly.dropna(subset=['month_num'])
fatalities_monthly['month_num'] = fatalities_monthly['month_num'].astype(int)
fatalities_monthly['month_name'] = fatalities_monthly['month_num'].apply(lambda m: calendar.month_name[m])

plt.figure(figsize=(14, 6))
plt.plot(fatalities_monthly['month_name'], fatalities_monthly['direct_deaths'],
         marker='o', linewidth=2, label='Direct Deaths')
plt.plot(fatalities_monthly['month_name'], fatalities_monthly['indirect_deaths'],
         marker='s', linewidth=2, label='Indirect Deaths')
plt.title('Monthly Fatalities (Direct & Indirect) in 2011', fontsize=16, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Number of Deaths')
plt.xticks(rotation=45)
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Average Deaths per Event Type (Direct + Indirect)

query_avg_deaths = f"""
SELECT
    event_type,
    COUNT(*) AS event_count,
    SUM(DEATHS_DIRECT + DEATHS_INDIRECT) AS total_deaths,
    AVG(DEATHS_DIRECT + DEATHS_INDIRECT) AS avg_deaths_per_event
FROM {storm_table_name}
GROUP BY event_type
HAVING total_deaths > 0
ORDER BY avg_deaths_per_event DESC
LIMIT 15
"""

avg_deaths_df = pd.read_sql(query_avg_deaths, conn)

plt.figure(figsize=(14, 7))
bars = plt.barh(avg_deaths_df['event_type'], avg_deaths_df['avg_deaths_per_event'],
                color=plt.cm.inferno(np.linspace(0, 1, len(avg_deaths_df))))

plt.title('Top 15 Deadliest Event Types (Average Deaths per Event)', fontsize=16, fontweight='bold')
plt.xlabel('Average Deaths per Event')
plt.ylabel('Event Type')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)

# Add value labels
for bar in bars:
    width = bar.get_width()
    plt.text(width + 0.01, bar.get_y() + bar.get_height()/2,
             f'{width:.2f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
# Damage Frequency by Event
query_damage_likelihood = f"""
SELECT
    event_type,
    COUNT(*) AS total_events,
    SUM(CASE
        WHEN damage_property REGEXP '[0-9]' OR damage_crops REGEXP '[0-9]' THEN 1
        ELSE 0
    END) AS damaging_events
FROM {storm_table_name}
GROUP BY event_type
HAVING total_events > 50
"""

damage_likelihood_df = pd.read_sql(query_damage_likelihood, conn)
damage_likelihood_df['percent_damaging'] = (
    damage_likelihood_df['damaging_events'] / damage_likelihood_df['total_events']
) * 100

# Sort by percent_damaging
damage_likelihood_df = damage_likelihood_df.sort_values(by='percent_damaging', ascending=False)

# Plot
plt.figure(figsize=(14, 7))
bars = plt.barh(damage_likelihood_df['event_type'], damage_likelihood_df['percent_damaging'],
                color=plt.cm.cividis(np.linspace(0, 1, len(damage_likelihood_df))))

plt.title('Damage Frequency by Event', fontsize=16, fontweight='bold')
plt.xlabel('% of Events That Cause Damage')
plt.ylabel('Event Type')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)

# Value labels
for bar in bars:
    width = bar.get_width()
    plt.text(width + 0.5, bar.get_y() + bar.get_height()/2,
             f'{width:.1f}%', va='center', fontsize=10)

# Show plot
plt.tight_layout()
plt.show()